In [ ]:
#   for data fetching
!pip install opendatasets


In [ ]:
import opendatasets as od
od.download("kaggle.com/datasets/hopesb/student-depression-dataset")


Skipping, found downloaded files in "./student-depression-dataset" (use force=True to force download)


In [ ]:
import pandas as pd
data=pd.read_csv("/content/student-depression-dataset/Student Depression Dataset.csv")
data.describe()
data.shape # tells rows and coloumns
# print(data.isnull().sum())
print(data.duplicated().sum())
data = data.drop_duplicates()
# print(data.duplicated().sum())
#no redundant data but missing data in column financial stress

0


In [ ]:
missing_vals=['Financial Stress']
for val in missing_vals:
  print(f"Number of missing values in {val} before handling:", data[val].isnull().sum())
  data.dropna (subset=[val], inplace=True)
  print(f"Number of missing values in {val} after handling:", data[val].isnull().sum())
  # removing mssing data

Number of missing values in Financial Stress before handling: 3
Number of missing values in Financial Stress after handling: 0


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer # Import SimpleImputer
import pandas as pd

df = pd.read_csv("/content/student-depression-dataset/Student Depression Dataset.csv")
columns_name = ['id', 'Gender', 'Age', 'City', 'Profession', 'Academic Pressure', 'Work Pressure',
              'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration', 'Dietary Habits',
              'Degree', 'Have you ever had suicidal thoughts?', 'Work/Study Hours',
              'Financial Stress', 'Family History of Mental Illness', 'Depression']
df.columns = columns_name
categorical_cols = ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits',
                  'Degree', 'Have you ever had suicidal thoughts?', 'Family History of Mental Illness']
# Removed extra spaces in ' Degree' and 'Have you ever had suicidal thoughts ?'
# One-Hot Encoding using pandas get_dummies
X = pd.get_dummies(df.drop(columns=['Depression']), columns=categorical_cols, drop_first=True,dummy_na=False)
# Applying one-hot encoding to X (features) and dropping the target column , dummy_na=False to avoid creating NaN
y = df['Depression']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='most_frequent') # Using 'most_frequent' for categorical features
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("knn classifier")
print(accuracy)

gnb=GaussianNB()
gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print("guassian classifier")
print(accuracy)

knn classifier
0.538075613689303
guassian classifier
0.7905393298691991


In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        # Initialize weights and biases
        np.random.seed(42)
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size) * 0.01
        self.bias_output = np.zeros((1, self.output_size))

        # Adam optimizer parameters
        self.m_wih = np.zeros_like(self.weights_input_hidden)
        self.v_wih = np.zeros_like(self.weights_input_hidden)
        self.m_bh = np.zeros_like(self.bias_hidden)
        self.v_bh = np.zeros_like(self.bias_hidden)
        self.m_who = np.zeros_like(self.weights_hidden_output)
        self.v_who = np.zeros_like(self.weights_hidden_output)
        self.m_bo = np.zeros_like(self.bias_output)
        self.v_bo = np.zeros_like(self.bias_output)
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.t = 0

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = self.sigmoid(self.output_layer_input)
        return self.predicted_output

    def backward(self, X, y):
        self.t += 1

        # Output layer error and gradient
        error_output = self.predicted_output - y.values.reshape(-1, 1)  # Convert y to NumPy array before reshaping
        gradient_output = error_output * self.sigmoid_derivative(self.predicted_output)

        # Hidden layer error and gradient
        error_hidden = np.dot(gradient_output, self.weights_hidden_output.T)
        gradient_hidden = error_hidden * self.sigmoid_derivative(self.hidden_layer_output)

        # Gradients for weights and biases
        grad_who = np.dot(self.hidden_layer_output.T, gradient_output)
        grad_bo = np.sum(gradient_output, axis=0, keepdims=True)
        grad_wih = np.dot(X.T, gradient_hidden)
        grad_bh = np.sum(gradient_hidden, axis=0, keepdims=True)

        # Adam optimizer updates
        self.m_who = self.beta1 * self.m_who + (1 - self.beta1) * grad_who
        self.v_who = self.beta2 * self.v_who + (1 - self.beta2) * (grad_who ** 2)
        m_who_hat = self.m_who / (1 - self.beta1 ** self.t)
        v_who_hat = self.v_who / (1 - self.beta2 ** self.t)

        self.m_bo = self.beta1 * self.m_bo + (1 - self.beta1) * grad_bo
        self.v_bo = self.beta2 * self.v_bo + (1 - self.beta2) * (grad_bo ** 2)
        m_bo_hat = self.m_bo / (1 - self.beta1 ** self.t)
        v_bo_hat = self.v_bo / (1 - self.beta2 ** self.t)

        self.m_wih = self.beta1 * self.m_wih + (1 - self.beta1) * grad_wih
        self.v_wih = self.beta2 * self.v_wih + (1 - self.beta2) * (grad_wih ** 2)
        m_wih_hat = self.m_wih / (1 - self.beta1 ** self.t)
        v_wih_hat = self.v_wih / (1 - self.beta2 ** self.t)

        self.m_bh = self.beta1 * self.m_bh + (1 - self.beta1) * grad_bh
        self.v_bh = self.beta2 * self.v_bh + (1 - self.beta2) * (grad_bh ** 2)
        m_bh_hat = self.m_bh / (1 - self.beta1 ** self.t)
        v_bh_hat = self.v_bh / (1 - self.beta2 ** self.t)

        # Update weights and biases
        self.weights_hidden_output -= self.learning_rate * m_who_hat / (np.sqrt(v_who_hat) + self.epsilon)
        self.bias_output -= self.learning_rate * m_bo_hat / (np.sqrt(v_bo_hat) + self.epsilon)
        self.weights_input_hidden -= self.learning_rate * m_wih_hat / (np.sqrt(v_wih_hat) + self.epsilon)
        self.bias_hidden -= self.learning_rate * m_bh_hat / (np.sqrt(v_bh_hat) + self.epsilon)

    def compute_loss(self, y):
        epsilon = 1e-10  # Small value to prevent log(0)
        y = y.values.reshape(-1, 1)  # Reshape target to match predicted output
        predicted_output_clipped = np.clip(self.predicted_output, epsilon, 1 - epsilon)
        return -np.mean(y * np.log(predicted_output_clipped) + (1 - y) * np.log(1 - predicted_output_clipped))
# Load and preprocess dataset
col = ["id",'Gender','Age','City','Profession','Academic Pressure','Work Pressure','CGPA','Study Satisfaction','Job Satisfaction','Sleep Duration','Dietary Habits','Have you ever had suicidal thoughts ?','Work/Study Hours','Financial Stress','Family History','Depression']
df = pd.read_csv('/content/student-depression-dataset/Student Depression Dataset.csv', names = col)

# Target and features
target = 'Depression'  # Replace with actual target column name
X = df.drop(columns=[target])
y = df[target]

# Encode categorical variables
categorical_cols = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical = encoder.fit_transform(X[categorical_cols])

# Combine numerical and encoded categorical data
X = X.drop(columns=categorical_cols)
X = np.hstack([X.values, encoded_categorical])

# Standardize numerical data
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Encode target variable
if y.dtype == 'object' or len(y.unique()) > 2:
    y = (y == y.unique()[1]).astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the neural network
input_size = X_train.shape[1]  # Number of features
hidden_size = 64  # Number of neurons in the hidden layer
output_size = 1  # Binary classification
learning_rate = 0.001
batch_size = 128

nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
epochs = 300

for epoch in range(epochs):
    nn.forward(X_train)
    nn.backward(X_train, y_train)

    if epoch % 50 == 0:
        loss = nn.compute_loss(y_train)
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate on test data
predicted_output_test = nn.forward(X_test)
predicted_binary = (predicted_output_test > 0.5).astype(int)
accuracy = np.mean(predicted_binary.flatten() == y_test)
print(f"Test Accuracy: {accuracy:.2f}")


# Calculate training accuracy
predicted_output_train = nn.forward(X_train)
predicted_train_binary = (predicted_output_train > 0.5).astype(int)
train_accuracy = np.mean(predicted_train_binary.flatten() == y_train)
print(f"Train Accuracy: {train_accuracy:.2f}")

Epoch 0, Loss: 0.6964
Epoch 50, Loss: 0.6657
Epoch 100, Loss: 0.6028
Epoch 150, Loss: 0.4955
Epoch 200, Loss: 0.4260
Epoch 250, Loss: 0.3891
Test Accuracy: 0.84
Train Accuracy: 0.85


In [36]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load and preprocess dataset
col = ["id", 'Gender', 'Age', 'City', 'Profession', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration', 'Dietary Habits',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours', 'Financial Stress',
       'Family History', 'Depression']
df = pd.read_csv('/content/student-depression-dataset/Student Depression Dataset.csv', names=col)

target = 'Depression'
X = df.drop(columns=[target])
y = df[target]

# Encode categorical variables
categorical_cols = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical = encoder.fit_transform(X[categorical_cols])

# Combine numerical and encoded categorical data
X = X.drop(columns=categorical_cols)
X = np.hstack([X.values, encoded_categorical])

# Encode target variable
if y.dtype == 'object' or len(y.unique()) > 2:
    y = (y == y.unique()[1]).astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the MLPClassifier
mlp = MLPClassifier(
    hidden_layer_sizes=(64,),  # Single hidden layer with 64 neurons
    activation='logistic',
    solver='adam',             # Adam optimizer
    learning_rate_init=0.0001,  # Learning rate
    max_iter=300,
    random_state=42
)

# Fit the model
mlp.fit(X_train, y_train)

# Evaluate on test data
y_pred_test = mlp.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Evaluate on training data
y_pred_train = mlp.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f"Train Accuracy: {train_accuracy:.2f}")




Test Accuracy: 0.85
Train Accuracy: 0.85

Classification Report (Test Data):
              precision    recall  f1-score   support

           0       0.83      0.79      0.81      2306
           1       0.86      0.88      0.87      3275

    accuracy                           0.85      5581
   macro avg       0.84      0.84      0.84      5581
weighted avg       0.85      0.85      0.85      5581

